# Biology 327 Lab: Week 7 (spring 2024)

## Processing sequencing reads and mapping to a reference genome

Remember these pro-tips:
- triple-click on a line to highlight the entire line to copy
- set up a split screen to put this notebook next to your terminal window

**New pro-tip for today:** you can use the up arrow to re-enter the last line of code that you ran. So if you forgot a character or had a different typo, just hit the up arrow and make the fix rather than retyping the whole thing!

## 0. Logging on to spydur:

First let's log into `spydur`. Update the code below with your netid and use it to log in by copying into terminal. If your key setup worked last week, you should not need to enter a password, but if you are prompted, enter your netid password (it won't show up as typing anything)

In [ ]:
ssh YOURNETID@spydur

## 1. Setting up folders
Now let's make new folders to work in for this week's lab. After last week, you should have a personal folder in the `~/shared_perickso/` directory on Spydur. (remember that `~` is a shortcut to `/home/YOURNETID`

Navigate to your personal folder now.

Use `mkdir` to make a subfolder called `Lab7`. 

change directories to your new `Lab7` folder.

In the end you should have a folder at `/home/YOURNETID/shared_perickso/YOURFIRSTNAME/Lab7` and be in that folder (replacing your netid and first name).

## 2. Introducting variables in Unix

Just like we can save different object as variable names in R, we can also save text, numbers, and files to variable names in UNIX. The advantage of doing this is that it can save you a lot of typing, and in some cases make complex operations much simpler. The formula is `variable_name=variable_value`. We can then refer back to those variables by typing `${variable_name}`. 

You could make a variable with your name like this: try running the code below in terminal

In [ ]:
name=Darwin
echo "${name} is my favorite scientist!"

If you changed "Darwin" to something else like "Marie-Claire King", it would instead print whatever word you had saved in the sentence. This seems trivial now but will become a useful skill as we start to build more complex code

## 3. Set up an interactive job
My lab (the Erickson lab) "owns" one computer that is part of the network of computers that makes up the **Spydur** computing cluster. We will be running our jobs on that computer.  To ensure that everyone's jobs are able to run, we will tell the computer how many cores of computing power and how much memory we need. Copy the code below and run it in terminal exactly as-is so that all your future commands run on my branch of `spydur`. This will give us each 20GB of memory on 3 computing cores to run our current operations for the next 3 hours. 

In [ ]:
srun --pty -t 3:00:00 --mem=20G --partition erickson --ntasks-per-node=3 bash

## 4. Make a variable to save our working directory

Now we'll save the name of the Lab7 folder as our "working directory". The working directory is where you have files and run code for the current analysis. We will save the directory as a variable name.  That way, we can just type `${WD}` instead of the full directory for everything we want to do. Update the code below with with the path to your Lab7 folder and run it. It should print back the location of `${WD}`. Anytime you want to use a path to your current working directory, you can type `${WD}` insteads of `/home/YOURNETID/shared_perickso/YOURFIRSTNAME/Lab7`. See how much typing that will save you? Enter your WD below, then use `echo ${WD}` to confirm that it saved correctly. 

In [ ]:
#provide path to your working directory after WD= below
WD=
echo ${WD}

Now use a command from last week to change directories to your new working directory, **using the shortcut variable** you just made above. Enter the code below and then copy and paste it to run in terminal. Then confirm you are working in the correct directory with a different command from last week. 


In [ ]:
#your code here


## 5. Finding the right files
We left off last week after looking at FastQ and FastA files.












**Question: What is the difference between the two file types?  What information is stored in each?**

**Answer here**

This week, we are going to take some of our sequencing reads from fastq files and learn how to process them and map them to a reference genome in a fasta file.


Use the command below to look at the file in `shared_perickso` named `samples.txt`. The`-S` after the `less` will make the columns display in a more orderly fashion.

In [ ]:
less -S ~/shared_perickso/samples.txt

This file contains all of the information about all of the _Z. indianus_ pooled sequencing libraries my lab has sequenced, sorted by the collection date. Here is the metadata for that file: 
- Pool: The name of the sequencing library. 
- Location: The location flies were collected:
    - CM = Carter Mountain, Charlottesville, VA
    - MIA = University of Miami, FL
    - FL = Fruit and Spice Park, Miami, FL
    - BGFL = Botantical Garden, Miami, FL
    - HPO = Hanover Peach Orchard, Richmond, VA
    - HCGA = Hillcrest Orchard, Ellijay, GA
    - CK = Cross Keys Orchard, Harrisonburg, VA
    - MA = Acton, MA (flies combined from multiple orchards)
    - CT = Lyman Orchards, Middleton, CT
    - PA = Linvilla Orchards, Media, PA
    - CH = Chiles Orchards, Crozet, VA
- Year: The year of sampling
- Sample Date: The day flies were collected
- females: The number of female flies in the pool
- males: The number of male flies in the pool
- total: The total number of flies in the pool **this is the final number in the row and is important for a question below**

Note that the columns don't display super nicely so the last three numbers are females, males, and total, regardless of how they are shifted on the page. 

**Question: What are the names of the two samples that you and your partner will be comparing? The name should be "ZP_" followed by a number**

**Answer here**


**Question: How many flies were squished up and pooled together for each of your samples?**

**Answer here**



**Question: How might the number of flies in the sample affect the accuracy of our allele frequency estimates? (which is generally more accurate, larger samples or smaller samples?)**

**Answer here**

**(Challenge!) Question: When sequencing is performed, you typically load a fixed amount of DNA onto the flow cell. Given that information, will the number of flies in the sample affect how many sequencing reads you get? Why or why not?**

**Answer here**

Each partner will process one of your two samples for steps 6-10. Decide now with your partner who is going to process which sample.
 
**Question: which of your two samples will you be working with going forward?**

**Answer here**

## 6. Looking at the real fastq files

Our fastq files have a lot of data in them: remember that we calculated last week that we need approximately 100,000,000 sequencing reads to sequence the genome 100 times to estimate allele frequencies!. The raw data are stored in a compressed form, indicated by a `.gz` at the end of the file.

Change directories to `/shared_perickso/fastq`. Then change directories to the folder for your sample. Use `ls -lh`  (those are both lower-case letter Ls) to see the names and sizes of your samples. G=gigabytes, M=megabytes, K=kilobytes. 


**Question: How big are the raw data files for your samples?**

**Answer here**


One of these compressed files might be bigger than your monthly cell phone data usage (the amount of data in all the videos and music you stream), which means they can take a long time to process. I ran a program that randomly downsampled the fastq files so that only 0.5% of sequencing the reads were retained. In other words, it kept one in every 200 of the original sequence reads from the machine. It was run equally on both the read_1 and read_2 files so that the same pairs of reads were retained in each file. So, we have 0.5% of the paired-end reads from the original sample to work with. These reads are saved in `/shared_perickso/dfastq` (the "d" is for "downsampled"). Make sure you use this folder or your next steps will take a very long time to run!

**Question: Navigate to `shared_perickso/dfastq` which contains the smaller files. Use a command from last week (or your cheat sheet to determine how many lines are in the "downsampled" files for your sample (make sure to run it on both the read_1 and read_2 files) and report those numbers** 

**Answer read 1**

**Answer read 2**


**Question: Given how sequences are represented in a `fastq` file, how many unique sequences does each fastq file have? (hint: remember that it takes more than one line to store the sequencing information**

**Answer here** 

**Question: If each file contains only one in every 200 reads from the starting file, how many total reads did you have? (count both read 1 and read 2!) How does that compare to the approximate number of reads you calculated last week? ** 

**Answer here**

## 7. Merging reads

Recall that we are using 150 bp, paired-end reads, which are sequenced from both ends of the same molecule.

**Question: Why might paired end reads occasionally have overlapping sequence? What would have to be true about the molecule of DNA that was sequenced for this to happen?**

**Answer here**

Because some reads might overlap, the sequence in the overlapping part represents duplicated data that is not a unique sequence. Therefore, we wouldn't want to count both of the overlapping reads as separate counts when we calculate our allele frequencies because we'd be double counting some information from the same molecule. We need to get rid of this duplication by merging the reads together. Essentially, the computer finds the overlap, if it exists, and comibines the two shorter overlapping reads into one long read that can then be aligned to the genome. So if you had read 1 as `ACTGGGGA` and read 2 was `GGGGATCTCA` the computer would combine the overlapping `GGGGA` to make a single read of `ACTGGGGATCTCA`

The software that we are going to use to merge our reads is called `bbmap`. The specific program we are using is called `merge.sh`. The program takes two input files, the `read_1` and `read_2` fastq files. It finds any occasions where the two reads overlap, and merges them into a single read. The merged reads get saved into a new file. The reads that don't overlap remain the same. 

It has two input files that it will use:
 - `in1`: The file containing the downsampled read_1 reads (from the `dfastq` folder)
 - `in2`: The file containing the downsampled read_2 reads (from the `dfastq` folder)
 
It also has three output files that it will generate; we will save all of these in your `Lab7` folder.
 - `out`: The merged reads (combining overlapping read 1 and read 2 reads into a single read)-should be saved in your `Lab7` folder
 - `outu1`: The unmerged read 1 reads (ie the read 1 reads that didn't overlap with their paired end and remain the same) -should be saved in your `Lab7` folder
 - `outu2`: The unmerged read 2 reads (ie the read 2 reads that didn't overlap with their paired end and remain the same) - should also be saved in your `Lab7` folder
 


**Question** What does the program need 2 input files and 3 output files?

**Answer here**

Before we run the program we are going to set up all of our file names as variables. Just like careful pipetting is critical to the success of a wet lab, proper file naming is going to be critical in this and future lab! We will assign each file name as a variable. `Samplename `refers to the ZP_# name of your pool. Please do not actually name your file "samplename"! Your input files already have names, so you will just need to make up names for your output files
 
How to name your files to save them in your Lab7 folder: (replace `samplename` with your ZP_ sample number)
 - `out`: `samplename.merged.fq`
 - `out1`: `samplename_1.unmerged.fq`
 - `out2`: `samplename_2.unmerged.fq`
 
Set up the names of all your files in the code block below. Because different files are in in different folders, you'll need to supply the full path (the location with all the folders) for each input and output file. **It is essential that you save the output files in your Lab7 folder-you'll have issues downstream if they aren't in the right place!** Don't forget you can use `${WD}` as a shortcut to this location! Once you have all the file names entered, copy this code to the terminal to save the variable names. 


In [ ]:
in1=
in2=
out=
out1=
out2=


Copy and paste all 5 lines above into terminal to save all the file names as variables. You may have to hit enter one extra time to make all 5 lines run.

Then run the code below to do the merging; if you did everything correct above, you shouldn't need to change anything. Note that we are using the variable names that we set up above to tell the computer which files to use for the input and output. If we had to type all of those file names, this would be a very long bit of code.

In [ ]:
bbmerge.sh \
    in1=${in1} \
    in2=${in2} \
    out=${out} \
    outu1=${out1} \
    outu2=${out2} \
    strict=t

**Question: The total number of merged and unmerged reads should add up to our original number of reads (which you calculated in step 6. Use the appropriate command from Table 2 to count the number of lines in your merged file and your unmerged read 1 file. Report all the numbers here. Do the numbers make sense?**

**Answer here**


## 8. Mapping the reads

Now it's time to map or align your fastq files to the reference genome. The program we will use is called `bwa`, which stands for Burrows-Wheeler aligner; named after the two people who invented the algorithm that it uses. Recall that when we mapped reads by hand, you scanned the DNA sequence of the reference genome looking for similarities between it and your sequencing read. This is essentially what the `bwa` program does, but it has to scan all ~150,000,000 bases of our reference genome and potentially deal with frequent mismatches. 

Each person will run BWA twice: once on their merged reads, and once on their unmerged (still paired-end reads). 
 


**Question: how do you think the mapping algorithm works differently when it is using single reads (the merged reads) vs paired reads (the read_1 and read_2 files)? Remember the physical relationship between paired end reads.**

**Answer here**

The overall syntax of how we run the program is

`bwa mem -M -t 3 <reference_genome> <fastq_file> <(fastq_file_2)> > <output_file.sam>`

 - `mem` is the specific algorithm the computer uses to map the reads to the reference genome
 - `-M` is an option that makes the output files compatible with a program we will use in the future
 - `-t 3` tells the computer to use 3 computing cores (the number you have reserved from the previous step
 - `reference_genome` is the fasta file that contains our reference genome file (we looked at it last week)
 - `fastq_file` is the reads that you want to map: these will be the outputs from our merging steps!
 - `fastq_file_2` is the optional paired end data for what you want to map (not applicable for the merged reads)
 - `>` means "save the output to the file name after this symbol". This is a very common way to save files in Linux
 - `output_file` is the name of the file we will save the mapping data to. It has to end in ".sam". Ideally it will have the same name as the input file but end in .sam instead of .fastq
 
 
Below you will need to set up the code to run the merged and paired end files through bwa. The lines that have a # in front of them are notes and will not be read by Linux. The lines without a # are the actual code. The second step is going to take 10-15 minutes to run most likely, so you can move on and start working on the next section once your code is running, then come back to answer the questions below about the output of bwa.

In [ ]:
#set up the code below to map the merged reads. 

#path to the file with the reference genome (the fasta file we looked at last week in `~/shared_perickso`)
ref=

#file you want to save to. should be saved in your `Lab7 folder` and named `ZP_#_merged.sam` (replace the # with your sample #)
merged_mapped=


In [ ]:
#below is the code that will actually run the mapping for the merged reads, once you've run the above
bwa mem -M -t 3 $ref ${out} > $merged_mapped

Most of our reads are unmerged, so this next step is going to take longer, after you start the `bwa  mem` step you can proceed to the questions below while the code is running

In [ ]:
#now set up the code below to map the unmerged reads--the ones that are still separate as read 1 and read 2

#file that you want to save to. shouldbe saved in your `Lab7` folder named `ZP_#_merged.sam` (replace the # with your sample number)
unmerged_mapped=

In [ ]:
#use this code to run the mapping for the unmerged reads--you can expect this to take 10-15 minutes. 
bwa mem -M -t 3 $ref ${out1} ${out2} > $unmerged_mapped

**Question: While your code is running, look at your output .sam file for the merged reads which should be done (hint: use `less -S <filename>` to look at them in an orderly way that uses tabs to organize the columns). What key information does the sam file appear to contain?**

**Answer here**

**Question: While you are still waiting, read about the sam file format here: https://en.wikipedia.org/wiki/SAM_(file_format). What column will tell you where in the genome the sequence was aligned to?**

**Answer here**

**Question What column would be important if you wanted to identify reads that were a poor match to the reference genome?**

**Answer here**

Look at the output that printed to your screen after running each job:

**Question: how long did it take to run the two jobs?** 

**Answer here**


**Question: these reads where downsampled to 0.5% of their original read depth; how long would it have taken to run the whole file this way?**
**Answer here**

**Question: look at your output .sam files (hint: use `less -S <filename>` to look at them in an orderly way that uses tabs to organize the columns). What key information does the sam file appear to contain?**
**Answer here**

**Question: these reads where downsampled to 0.5% of their original read depth; how long would it have taken to run the whole file this way? (this is why I have to process the full dataset behind the scenes for you!)**

**Answer here**

## 9. Sorting, filtering, and converting to a bam file
We are so close to being done! We've mapped our reads to the genome, but we need to sort them and save them as a new tuype of file. Sorting puts all the mapped reads in order along the reference genome. Sorting is required for many downsteram steps. This step will also convert our sam files into a binary format file called a bam file. The information remains the same, but it is compressed and easier for the computer to work with. However, once it is in binary format, it is no longer human-readable, so it will just look like random characters if you tried to look at it with the `less` command.

This is also the step where we can remove poorly mapped reads by setting a quality filter. The `-q 40` option below means that we will remove anything with a mapping quality lower than 40. This is the first of many important filtering steps that will happen! 

You will need to repeat the sorting and filtering steps for both your merged and unmerged sam files. 

Your input file will be the output from the mapping step above. oOu will need to make up a new filename for the output file. You should include the word "sorted" in your sorted file name, so it will be something like `ZP_#_sorted.bam`. It's critical to have the `.bam` in the output file name so that the program saves the data in the correct format. Make sure that everything still gets saved in your Lab7 folder! 

In [ ]:
#provide the output named of your merged, sorted bam file using the guidelines above
merged_sorted=

#then run this command:
samtools sort -b -q 40 --threads 3  $merged_mapped> $merged_sorted

#now provide an output name for the unmerged, sorted bam
unmerged_sorted=

#then run this command:
samtools view -b -q 40 --threads 3  $unmerged_mapped > $unmerged_sorted


**Question: Take a look at one of the bam files you just made with `less`. What does it look like inside?**

**Answer here**

## 10. Combine files
Now that our bam files are created and sorted, we can combine them into a final bam file containing all of our mapped reads (from the merged and unmerged). The syntax for this command is different; instead of using a > to tell the program where to save the file, the name of the **output** file is the first argument after the command

`samtools merge <output_file> <input_file_1> <input_file_2>`

**Your `output_file` should be named `ZP_#_final.bam` and saved in your Lab7 folder (`${WD}`--I'll be checking to make sure this file is here!** 

This is your final little check to see if you understand how some of the code worked above. Use the example above to create a line of code below that will combine your bam files that you made in step 9 into a final bam file named and located as described above.


In [ ]:
#use the syntax explanation above to write your command below,  then run it. Remember that your input files are the sorted files you created above! 


**Final assignment: To check your understanding of what we did today, write a 3-4 sentence "Methods" section outlining what we did with our raw data to merge, map, filter and sort it. You do not need to include specifics like the names of files or folders, but you should name each program we used and say what it did. If time is running short, you can complete this part outside of the lab period and turn it in as soon as you are able.**

**Paragraph here*



When you are done, hit File> Print. Then print the resulting window to a PDF and upload to Blackboard for Lab 7. 